In [89]:
import sys
import os
sys.path.insert(0, '/u/monitors/python_bin/')
sys.path.insert(0, os.path.expanduser("~")+'/python_bin_private/')
import pandas as pd
import string
from datetime import datetime, timedelta
import numpy as np
import paramiko
import da_functions as oqm
import cred
import sql_connections as sql

from sn_extractor import extract_data
import sn_extractor  
admin_conn = sn_extractor.connect_to_snowflake('ADMIN_WW_DEMQRA@micron.com', 'ADMIN_WW_DEMQRA_DEFAULT', 'Micron123')
conn = sql.connect_to_snowflake()

In [8]:
data = pd.read_csv('FG_Version.csv')
data.head()

,Material,Material Description,Material Group,Class,Plant,Storage Type,Storage Location,Batch,Stock Category,Unrest. Qty,...,VIRTUAL_HUB,Component Qty,260 comp qty_n,260 mod qty_n,Remark_n,FID_RISK_COUNT,MOD_RISK_COUNT,Pullfrom SN,Remark,SN Match
0,468890,MTC8C1084S1SC48BA1,2,MODULE,SG31,ASR,FG02,DPAX77SA03,NaN,100,...,8005531-ACER,800,0,0,Good,NaN,NaN,NaN,NaN,NaN
1,468890,MTC8C1084S1SC48BA1,2,MODULE,SG31,ASR,FG02,DPAX922A01,NaN,100,...,8005531-ACER,800,0,0,Good,NaN,NaN,NaN,NaN,NaN
2,468890,MTC8C1084S1SC48BA1,2,MODULE,SG31,ASR,FG02,DPAX923A01,NaN,100,...,8005531-ACER,800,0,0,Good,NaN,NaN,NaN,NaN,NaN
3,468890,MTC8C1084S1SC48BA1,2,MODULE,SG31,ASR,FG02,DPAX924A01,NaN,100,...,8005531-ACER,800,0,0,Good,NaN,NaN,NaN,NaN,NaN
4,468890,MTC8C1084S1SC48BA1,2,MODULE,SG31,ASR,FG02,DPAX925A01,NaN,100,...,8005531-ACER,800,0,0,Good,NaN,NaN,NaN,NaN,NaN


In [92]:
cmd = f'''
SELECT FUSEID FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_FWOVQQREHS" LIMIT 10 
'''
temp=pd.read_sql(cmd,conn).rename(columns=str.upper)
temp.drop_duplicates(keep='first', inplace=True)
temp.head()

,FUSEID
0,691415K:21:P07:23
1,691415K:24:P07:11
2,689448K:06:N09:22
3,689448K:09:P08:30
4,689448K:06:P04:21


In [15]:
temp[temp['ASSEMBLY_FACILITY']=='PTI XIAN']

,BATCH,AVAILABLE_STOCK,ASSEMBLY_FACILITY,MODULE_FORM_FACTOR
10,DPAXLAWA0F,100,PTI XIAN,RDIMM
11,DPAXLAWA0G,100,PTI XIAN,RDIMM
12,DPAXLAWA0H,100,PTI XIAN,RDIMM
13,FH11BD8.7Q,19,PTI XIAN,RDIMM


In [18]:
with open('BATCH.txt') as f:
    lines = f.readlines()
lines = [x.replace('\n','') for x in lines]

In [25]:
compare = pd.DataFrame(columns=[lines[0]], data=lines[1:])

In [83]:
master_attr = 'TMP_TABLE_WVPYSPVOEO'
fg_tab = 'TMP_TABLE_DUZVTYHBBU'

cmd = f'''
SELECT DISTINCT(ASM) FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_VVUVEWPLME" 
'''
temp1=pd.read_sql(cmd,conn).rename(columns=str.upper)
temp1.drop_duplicates(keep='first', inplace=True)
temp1.head()



,ASM
0,HD33501.BJ
1,HD33501.8X
2,HD33501.BW
3,HD33501.DX
4,HD33501.9Y


In [87]:


cmd = f'''
SELECT DISTINCT FUSEID
FROM "PROD_WW_BEMFG_ODS"."TTE"."TTE_2DID_COMP_HISTORY" TTE
WHERE ASM_BOX_ID IN ((SELECT ASM FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_VVUVEWPLME"))
'''
temp1=pd.read_sql(cmd,conn).rename(columns=str.upper)
temp1.drop_duplicates(keep='first', inplace=True)
temp1.head()


,FUSEID
0,685997K:03:P09:14
1,685997K:04:N04:33
2,685997K:04:N08:14
3,685997K:03:P11:27
4,685997K:02:P09:16


In [88]:
#upload tmp table
import sn_extractor  
admin_conn = sn_extractor.connect_to_snowflake('ADMIN_WW_DEMQRA@micron.com', 'ADMIN_WW_DEMQRA_DEFAULT', 'Micron123')
mod_fg_asm_tab = sn_extractor.upload_tmp_table(temp1,admin_conn)
print(mod_fg_asm_tab)
print('RISK FID')

0 70938672
2000000 70938672
4000000 70938672
6000000 70938672
8000000 70938672
10000000 70938672
12000000 70938672
14000000 70938672
16000000 70938672
18000000 70938672
20000000 70938672
22000000 70938672
24000000 70938672
26000000 70938672
28000000 70938672
30000000 70938672
32000000 70938672
34000000 70938672
36000000 70938672
38000000 70938672
40000000 70938672
42000000 70938672
44000000 70938672
46000000 70938672
48000000 70938672
50000000 70938672
52000000 70938672
54000000 70938672
56000000 70938672
58000000 70938672
60000000 70938672
62000000 70938672
64000000 70938672
66000000 70938672
68000000 70938672
70000000 70938672
TMP_TABLE_FWOVQQREHS
RISK FID


In [76]:
A = list(compare['BATCH'].drop_duplicates())

In [77]:
B = list(temp['ASM'].drop_duplicates())

In [78]:
print(f'New ASM BOX is {len(A)}')


21117

In [79]:
len(B)

5314

In [35]:
data = [x for x in A if x not in B]
len(data)

15803

In [36]:
## check all PTI FID
cmd = f'''
    SELECT * FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_SZOCOOPDIP"
'''
print(cmd)

com_pti_res=pd.read_sql(cmd,conn).rename(columns=str.upper)
com_pti_res.drop_duplicates(keep='first', inplace=True)
com_pti_res.head()


    SELECT * FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_SZOCOOPDIP"



,BOX,FID
0,DQFQGGP.41,680029K:24:N03:07
1,DQFQGGP.41,680029K:12:P08:07
2,DQFQGGP.41,680240K:23:P07:24
3,DQFQGGP.41,683129K:17:P04:34
4,DQFQGGP.41,683129K:01:N04:12


In [38]:
com_pti_res.to_csv('QA_COMPONENT.csv', index=False)

In [ ]:
## check all PTI FID
cmd = f'''
    SELECT FUSEID
    FROM "PROD_WW_BEMFG_ODS"."TTE"."TTE_2DID_COMP_HISTORY" TTE
    WHERE ASM_BOX_ID IN ((SELECT ASM FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_VVUVEWPLME"))
'''
print(cmd)

com_pti_res=pd.read_sql(cmd,conn).rename(columns=str.upper)
com_pti_res.drop_duplicates(keep='first', inplace=True)
com_pti_res.head()

In [ ]:
## check all PTI FID
cmd = f'''
    SELECT FUSEID
    FROM "PROD_WW_BEMFG_ODS"."TTE"."TTE_2DID_COMP_HISTORY" TTE
    WHERE ASM_BOX_ID IN ((SELECT ASM FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_VVUVEWPLME"))
'''
print(cmd)

com_pti_res=pd.read_sql(cmd,conn).rename(columns=str.upper)
com_pti_res.drop_duplicates(keep='first', inplace=True)
com_pti_res.head()

In [43]:
cmd = '''
SELECT
    MA_ID,ATTR_ID,ATTR_VALUE
FROM
    PROD_WW_BEMFG_ODS.MAM.MA_ATTR
WHERE
    MA_ID IN ({})
AND ATTR_ID in ('CURRENT QTY')
'''
print(cmd)

batch2asm = extract_data(cmd,list(asm['ASM_BOX_ID'].drop_duplicates()), 'ASM_BOX_ID' )
batch2asm.head()



SELECT
    MA_ID,ATTR_ID,ATTR_VALUE
FROM
    PROD_WW_BEMFG_ODS.MAM.MA_ATTR
WHERE
    MA_ID IN ({})
AND ATTR_ID in ('CURRENT QTY')

0 5314
final_sql_cmd:   SELECT     MA_ID,ATTR_ID,ATTR_VALUE FROM     PROD_WW_BEMFG_ODS.MAM.MA_ATTR WHERE     MA_ID IN (SELECT DISTINCT ASM_BOX_ID FROM "ENG_MFG_WW_GQ_DEMQRA_DA"."DEMQRA_DA"."TMP_TABLE_BNWPHTMQBB") AND ATTR_ID in ('CURRENT QTY') 
Extracting data from snowflake ...
finish!
finally


,MA_ID,ATTR_ID,ATTR_VALUE
0,HD34000.HM,CURRENT QTY,42
1,HD34302.FY,CURRENT QTY,16439
2,HD34401.K4,CURRENT QTY,42
3,HD34301.GN,CURRENT QTY,1058
4,HD34200.99,CURRENT QTY,20301


In [61]:
batch2asm[batch2asm['MA_ID']=='HD33300.GV']

,MA_ID,ATTR_ID,ATTR_VALUE
8189,HD33300.GV,CURRENT QTY,16821
8231,HD33300.GV,CURRENT QTY,16853
10824,HD33300.GV,CURRENT QTY,17
12348,HD33300.GV,CURRENT QTY,16846
18023,HD33300.GV,CURRENT QTY,8


In [62]:
batch2asm['ATTR_VALUE']=batch2asm['ATTR_VALUE'].astype(int)

In [63]:
max_qty = batch2asm[['MA_ID','ATTR_VALUE']].groupby('MA_ID')['ATTR_VALUE'].idxmax()

In [64]:
max_qty = max_qty.reset_index()

,MA_ID,ATTR_VALUE
0,HD33300.GV,8231
1,HD33300.L5,20128
2,HD33300.LQ,23852
3,HD33300.NS,18905
4,HD33300.Z8,979
...,...,...
5309,HD34501.2P,12419
5310,HD34501.3N,11417
5311,HD34501.ZW,3751
5312,HD34502.01,19389


In [40]:
asm = pd.read_excel('PTI_Impact_Fid By (ASM_BOX_ID).xlsx', sheet_name='PTI_Impact_Fid By (ASM_BOX_ID)')

In [69]:
data = pd.merge(asm, batch2asm, left_on='ASM_BOX_ID', right_on='MA_ID')

In [70]:
data['diff'] = data['N Rows']-data['ATTR_VALUE']

In [75]:
data[data['diff']<10].to_csv('cacu_asm.csv', index=False)

In [41]:
list(asm['ASM_BOX_ID'].drop_duplicates())

,ASM_BOX_ID,N Rows
0,HD33300.GV,16848.0
1,HD33300.L5,15984.0
2,HD33300.LQ,16612.0
3,HD33300.NS,17441.0
4,HD33300.Z8,10315.0
...,...,...
5309,HD34501.3N,13290.0
5310,HD34501.ZW,10006.0
5311,HD34502.01,10057.0
5312,HD34502.03,10038.0


In [ ]:
data2 = pd.read_excel('PTI_Impact_Fid.xlsx', sheet_name=)

In [ ]:
#upload tmp table
import sn_extractor  
admin_conn = sn_extractor.connect_to_snowflake('ADMIN_WW_DEMQRA@micron.com', 'ADMIN_WW_DEMQRA_DEFAULT', 'Micron123')
mod_fg_asm_tab = sn_extractor.upload_tmp_table(batch2asm,admin_conn)
print(mod_fg_asm_tab)
print('Batch2msn_fid_asm')